In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install osmnx==0.10
!pip install folium

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import glob
import re
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as sio
import pywt
import scipy.stats as stats
import datetime as dt
import itertools
import folium
from geopy import distance
import csv
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
%matplotlib inline

In [ ]:
pip install gmplot

     |████████████████████████████████| 164 kB 4.2 MB/s 


In [ ]:
# os.chdir('/content/drive/My Drive/Colab Notebooks/MAHE Innovation Competition')
os.chdir('/content/drive/My Drive/MAHE Innovation Competition')
datapath = os.getcwd()+'/Data/'
filename = 'june9_mysuru_bike.csv'
df = pd.read_csv(datapath+filename, skiprows = 0, sep = ',', index_col = False)
df.drop(df.columns[[0, 1, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 18]], axis=1, inplace=True)
df.columns = ['a_z', 'lat', 'lon', 't']
df['t'] = df['t'] - df['t'].values[0]

data = []
datafilename = 'segmented_output.csv'
startindex = 0
j = 0
coords_1 = (df.loc[startindex,'lat'], df.loc[startindex,'lon'])
for index in range(len(df)):
  subdata = []
  subdata.append(['Road segment', j])
  subdata.append(['Start latitude and longitude', coords_1[0], coords_1[1]])
  coords_2 = (df.loc[index,'lat'], df.loc[index,'lon'])
  d = distance.distance(coords_1, coords_2).m
  if d >= 100.0:
    subdata.append(['End latitude and longitude', coords_2[0], coords_2[1]])
    subdata.append(['Distance', d])
    subdata.append(['Time']+list(df.loc[startindex:index+1, 't'].values-df.loc[startindex, 't']))
    subdata.append(['Z_Acceleration']+list(df.loc[startindex:index+1, 'a_z'].values))
    data.append(subdata)
    if j == 0:
      writemode = "w"
    else:
      writemode = "a"
    with open(os.getcwd() + '/Data/' + datafilename, writemode, newline="") as f:
                writer = csv.writer(f)
                writer.writerows(subdata)
    startindex = index
    j = j+1
    coords_1 = (df.loc[startindex,'lat'], df.loc[startindex,'lon'])

df

In [ ]:
with open(os.getcwd() + '/Data/' + datafilename, 'rt') as f:
    reader = csv.reader(f)
    results = filter(lambda x: 'Z_Acceleration' in x, reader)
    # start_lat_long = filter(lambda x: 'Start latitude and longitude' in x, reader)
    # end_lat_long = filter(lambda x: 'End latitude and longitude' in x, reader)

    data_raw = {}
    data_normalized = {}
    data_length = []
    j = 0
    for line in results:
      a_z = np.array(line[1:None]).astype(np.float)
      data_length.append(len(a_z))
      data_normalized[j] = (a_z-np.mean(a_z))/np.std(a_z)
      j = j+1


data_ninterpolated = {k: np.interp(np.linspace(0, 1, np.max(data_length)),
                                   np.linspace(0, 1, data_length[k]), data)
              for k, data in data_normalized.items()}
[plt.plot(data) for data in data_ninterpolated.values()]
plt.show()

df_final = pd.DataFrame.from_dict(data_ninterpolated, 'index')
df_final
with open(os.getcwd() + '/Data/' + datafilename, 'rt') as f:
    reader = csv.reader(f)
    start_lat_long = filter(lambda x: 'Start latitude and longitude' in x, reader)
    startlatlongdf=pd.DataFrame(start_lat_long)

with open(os.getcwd() + '/Data/' + datafilename, 'rt') as f:
    reader = csv.reader(f)
    end_lat_long = filter(lambda x: 'End latitude and longitude' in x, reader)
    endlatlongdf=pd.DataFrame(end_lat_long)
endlatlongdf

In [ ]:
final = []
# coords_1 = (df.loc[startindex,'lat'], df.loc[startindex,'lon'])
for i in range(len(df_final)):
  # for index in range(len(df)):

  # final.append(['Road segment', data[i][0][1]])
  final.append(['Start latitude and longitude',data[i][1][1],data[i][1][2]])
  final.append(['End latitude and longitude',data[i][2][1],data[i][2][2]])
  final.append(["data"]+list(df_final.iloc[i,:].values))
df_cluster = pd.DataFrame(final)


from scipy.fftpack import fft, fftfreq, fftshift
# print(df_cluster.shape[0]-1)
group =[]
i=2
while(i!=len(df_cluster)+2):
  # print(i)
  group.append(fft(df_cluster.iloc[i,1:].values))
  i=i+3
len(group)



df_cluster

In [ ]:
finaldf=pd.DataFrame(group)
finaldf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7765,7766,7767,7768,7769,7770,7771,7772,7773,7774,7775,7776,7777,7778,7779,7780,7781,7782,7783,7784,7785,7786,7787,7788,7789,7790,7791,7792,7793,7794,7795,7796,7797,7798,7799,7800,7801,7802,7803,7804
0,0.183702-0.000000j,-57.331240-128.008575j,-258.272313-126.627953j,138.797184-46.963807j,39.101082+22.749301j,-31.265338-416.116339j,-281.056868-102.411571j,89.176195+414.177769j,372.228490+312.241739j,124.544206-148.379507j,188.039525-116.958732j,245.117154+26.670611j,156.415791-222.412475j,-254.257020-173.866726j,-424.823445-230.968977j,-213.346280+149.389998j,81.820915-15.948541j,287.221435-117.760221j,4.842490+5.697119j,-300.699945+15.366895j,345.202538+68.568214j,152.927404-62.508553j,-185.330638-398.354471j,-174.247198-101.408560j,87.819510-132.204974j,462.347875-39.478815j,31.495136-301.778200j,-155.356195-499.266261j,319.461919+353.262387j,322.512107+45.428862j,-134.015709-82.516936j,-405.407918-24.159630j,-5.401569+412.338994j,452.517862+188.970449j,382.370394-379.794039j,-280.578636-515.751257j,-33.144359+153.211296j,162.487719+324.477127j,243.697200-199.324960j,-210.004526-431.479731j,...,-241.871293-203.381218j,-210.004526+431.479731j,243.697200+199.324960j,162.487719-324.477127j,-33.144359-153.211296j,-280.578636+515.751257j,382.370394+379.794039j,452.517862-188.970449j,-5.401569-412.338994j,-405.407918+24.159630j,-134.015709+82.516936j,322.512107-45.428862j,319.461919-353.262387j,-155.356195+499.266261j,31.495136+301.778200j,462.347875+39.478815j,87.819510+132.204974j,-174.247198+101.408560j,-185.330638+398.354471j,152.927404+62.508553j,345.202538-68.568214j,-300.699945-15.366895j,4.842490-5.697119j,287.221435+117.760221j,81.820915+15.948541j,-213.346280-149.389998j,-424.823445+230.968977j,-254.257020+173.866726j,156.415791+222.412475j,245.117154-26.670611j,188.039525+116.958732j,124.544206+148.379507j,372.228490-312.241739j,89.176195-414.177769j,-281.056868+102.411571j,-31.265338+416.116339j,39.101082-22.749301j,138.797184+46.963807j,-258.272313+126.627953j,-57.331240+128.008575j
1,0.183952-0.000000j,-330.732525-108.494044j,-185.003571-29.951004j,-140.727679+94.636260j,-472.260496-36.692098j,776.744694-42.708049j,-484.440752-47.011464j,517.774751-62.476778j,-449.706564-122.326578j,83.367806+87.251000j,127.754169+114.114835j,55.110885-214.415778j,434.597366-117.233428j,-492.278991-65.420780j,392.795476+391.248891j,11.873122+219.961328j,68.057031-202.174587j,-217.354009-48.911072j,-596.522520+267.834461j,334.112103+219.947026j,90.543886+851.634460j,197.881032-568.636717j,-79.137680-87.416354j,47.141475+189.680338j,141.779370-93.749613j,578.650607+990.337774j,-496.368349-55.812997j,-383.439833-59.082841j,170.133916+40.796602j,141.113326+943.893603j,936.606122-455.845210j,109.647777-216.694529j,4.727918-254.273573j,-232.669141+443.076965j,569.750651-335.712067j,-247.901493-52.220121j,-115.619852+174.622173j,-341.011288-485.653736j,482.043987+707.486620j,106.945310-949.981708j,...,467.884321-277.903509j,106.945310+949.981708j,482.043987-707.486620j,-341.011288+485.653736j,-115.619852-174.622173j,-247.901493+52.220121j,569.750651+335.712067j,-232.669141-443.076965j,4.727918+254.273573j,109.647777+216.694529j,936.606122+455.845210j,141.113326-943.893603j,170.133916-40.796602j,-383.439833+59.082841j,-496.368349+55.812997j,578.650607-990.337774j,141.779370+93.749613j,47.141475-189.680338j,-79.137680+87.416354j,197.881032+568.636717j,90.543886-851.634460j,334.112103-219.947026j,-596.522520-267.834461j,-217.354009+48.911072j,68.057031+202.174587j,11.873122-219.961328j,392.795476-391.248891j,-492.278991+65.420780j,434.597366+117.233428j,55.110885+214.415778j,127.754169-114.114835j,83.367806-87.251000j,-449.706564+122.326578j,517.774751+62.476778j,-484.440752+47.011464j,776.744694+42.708049j,-472.260496+36.692098j,-140.727679-94.636260j,-185.003571+29.951004j,-330.732525+108.494044j
2,3.102769-0.000000j,243.366760+

# kmeans

In [ ]:
wcss=[]

for i in range(1,25):
    kmeans=KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0)
    kmeans.fit(np.abs(finaldf.iloc[:,:].values))
    wcss.append(kmeans.inertia_) #calculates the within cluster sum of square

plt.plot(range(1,25),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('wcss')
plt.show()

In [ ]:
x=np.abs(finaldf.iloc[:,:].values)
print(x.shape)
kmeans=KMeans(n_clusters=4,init='k-means++',max_iter=300,n_init=10,random_state=0)
y_means=kmeans.fit_predict(x)
for i in y_means:
  print(i,end=" ")

print("\n")

print(len(y_means==0))

startlatlongdf

(85, 7805)
2 1 2 3 3 1 3 3 3 3 0 3 3 3 3 3 3 1 2 3 3 3 2 3 1 2 1 2 1 0 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 0 2 2 1 1 0 2 1 1 1 1 3 1 1 2 2 1 2 0 1 2 2 1 1 1 1 2 2 1 1 1 1 1 1 0 

85


,0,1,2
0,Start latitude and longitude,12.288644999999999,76.59817
1,Start latitude and longitude,12.288732000000001,76.59723000000001
2,Start latitude and longitude,12.288824,76.596275
3,Start latitude and longitude,12.287931,76.59597
4,Start latitude and longitude,12.287905,76.59691
...,...,...,...
80,Start latitude and longitude,12.290231,76.59439
81,Start latitude and longitude,12.290341999999999,76.59533
82,Start latitude and longitude,12.290274,76.5963
83,Start latitude and longitude,12.290227,76.59724399999999


In [ ]:
#For plotting 1st cluster(0)

lats=startlatlongdf.iloc[(y_means==0),1].values
longs=startlatlongdf.iloc[(y_means==0),2].values

late=endlatlongdf.iloc[(y_means==0),1].values
longe=endlatlongdf.iloc[(y_means==0),2].values



latitude_list_s = list(lats.astype('float64'))
longitude_list_s = list(longs.astype('float64'))
latitude_list_e = list(late.astype('float64'))
longitude_list_e = list(longe.astype('float64'))

import gmplot

gmap3 = gmplot.GoogleMapPlotter(12.288644999999999,76.59817,18)

for i,j,k,l in zip(latitude_list_s,longitude_list_s,latitude_list_e,longitude_list_e):
  gmap3.scatter( [i,k],[j,l], '#FF0000', size = 10, marker = False )
  gmap3.plot([i,k],[j,l],'cornflowerblue', edge_width = 2.5)

In [ ]:
#For plotting 2nd cluster(1)

lats=startlatlongdf.iloc[(y_means==1),1].values
longs=startlatlongdf.iloc[(y_means==1),2].values

late=endlatlongdf.iloc[(y_means==1),1].values
longe=endlatlongdf.iloc[(y_means==1),2].values

latitude_list_s = list(lats.astype('float64'))
longitude_list_s = list(longs.astype('float64'))
latitude_list_e = list(late.astype('float64'))
longitude_list_e = list(longe.astype('float64'))

# import gmplot

# gmap3 = gmplot.GoogleMapPlotter(13.354924,74.798970,18)

for i,j,k,l in zip(latitude_list_s,longitude_list_s,latitude_list_e,longitude_list_e):
  gmap3.scatter( [i,k],[j,l], '#FF0000', size = 10, marker = False )
  gmap3.plot([i,k],[j,l],'yellow', edge_width = 2.5)


In [ ]:
#For plotting 3rd cluster(2)

lats=startlatlongdf.iloc[(y_means==2),1].values
longs=startlatlongdf.iloc[(y_means==2),2].values

late=endlatlongdf.iloc[(y_means==2),1].values
longe=endlatlongdf.iloc[(y_means==2),2].values

latitude_list_s = list(lats.astype('float64'))
longitude_list_s = list(longs.astype('float64'))
latitude_list_e = list(late.astype('float64'))
longitude_list_e = list(longe.astype('float64'))

# import gmplot

# gmap3 = gmplot.GoogleMapPlotter(13.354924,74.798970,18)

for i,j,k,l in zip(latitude_list_s,longitude_list_s,latitude_list_e,longitude_list_e):
  gmap3.scatter( [i,k],[j,l], '#FF0000', size = 10, marker = False )
  gmap3.plot([i,k],[j,l],'purple', edge_width = 2.5)

In [ ]:


import gmplot

gmap3 = gmplot.GoogleMapPlotter(12.288644999999999,76.59817,18)
i=13.3562646
j=74.7875994
k=13.3562374
l=74.7875751
gmap3.scatter( [i,k],[j,l], '#FF0000', size = 10, marker = False )
gmap3.plot([i,k],[j,l],'cornflowerblue', edge_width = 2.5)

In [ ]:
os.getcwd()

'/content'

In [ ]:

datapath = os.getcwd()+'/Data/'

In [ ]:
gmap3.draw(datapath+"Kmeans_4cluster_mysuru_bike.html")

In [ ]:
!pip install gmplot

In [ ]:
#For plotting 4th cluster(3)

lats=startlatlongdf.iloc[(y_means==3),1].values
longs=startlatlongdf.iloc[(y_means==3),2].values

late=endlatlongdf.iloc[(y_means==3),1].values
longe=endlatlongdf.iloc[(y_means==3),2].values

latitude_list_s = list(lats.astype('float64'))
longitude_list_s = list(longs.astype('float64'))
latitude_list_e = list(late.astype('float64'))
longitude_list_e = list(longe.astype('float64'))

# import gmplot ನಮಸ್ಕಾರ

# gmap3 = gmplot.GoogleMapPlotter(13.354924,74.798970,18)

for i,j,k,l in zip(latitude_list_s,longitude_list_s,latitude_list_e,longitude_list_e):
  gmap3.scatter( [i,k],[j,l], '#FF0000', size = 10, marker = False )
  gmap3.plot([i,k],[j,l],'orange', edge_width = 2.5)

In [ ]:
gmap3.draw( datapath+"Kmeans_4cluster_mysuru_bike.html" )
# gmap3.draw( datapath+"dummy1.html" )